## 1. Modify your Gaussian elimination code to obtain the L, U , and P matrices from the coefficient matrix of linear system AX = B.

In [3]:
def row_swap(M, i, j):
    mat = M.copy()
    mat[i] = M[j]
    mat[j] = M[i]
    return mat

def row_scale(M, row, scaler):
    M[row] = M[row]*scaler
    return M

def row_add(M, i, j, scaler = 1):
    mat = M.copy()
    mat[i] = mat[i]*scaler + mat[j]
    return mat

In [6]:
def gaussian_elimination_with_pivot(matrix):
    """
    Perform Gaussian elimination with partial pivoting on the given matrix.

    Parameters:
    - matrix: 2D NumPy array representing the matrix

    Returns:
    - L: Lower triangular matrix
    - U: Upper triangular matrix
    - P: Permutation matrix
    """
    rows, cols = matrix.shape
    L = np.eye(rows, dtype=float)
    U = matrix.astype(float)
    P = np.eye(rows, dtype=float)

    for pivot_row in range(min(rows, cols)):
        # Partial pivoting
        pivot_col = np.argmax(np.abs(U[pivot_row:, pivot_row])) + pivot_row
        if pivot_row != pivot_col:
            row_swap(U, pivot_row, pivot_col)
            row_swap(P, pivot_row, pivot_col)
            row_swap(L, pivot_row, pivot_col)

        # Make the pivot element 1
        pivot_value = U[pivot_row, pivot_row]
        L[pivot_row, pivot_row] = 1  # Set diagonal of L to 1
        U[pivot_row] /= pivot_value

        # Eliminate nonzero values below the pivot
        for i in range(pivot_row + 1, rows):
            multiplier = U[i, pivot_row]
            L[i, pivot_row] = multiplier
            U[i] -= multiplier * U[pivot_row]

    return L, U, P

In [7]:
# Example usage
A = np.array([[2, 5, 0, -4],
              [-4, -4, -3, 7],
              [-6, -3, -7, -6],
              [-1, 2, -6, 5]])

L, U, P = gaussian_elimination_with_pivot(A)

print("L matrix:")
print(L)
print("\nU matrix:")
print(U)
print("\nP matrix:")
print(P)


L matrix:
[[ 1.    0.    0.    0.  ]
 [-4.    1.    0.    0.  ]
 [-6.   12.    1.    0.  ]
 [-1.    4.5  -3.75  1.  ]]

U matrix:
[[ 1.          2.5         0.         -2.        ]
 [ 0.          1.         -0.5        -0.16666667]
 [-0.         -0.          1.         16.        ]
 [ 0.          0.          0.          1.        ]]

P matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## 2. Once L, U and P matrices are constructed, the solution X can be obtained as follows:
1. Compute the column vector P B.
2. Solve LY = P B for Y using forward substitution.
3. Solve U X = Y for X using back substitution.

In [13]:
def row_reduction(matrix):
    """
    Perform row reduction (Gaussian elimination) on the given matrix.

    Parameters:
    - matrix: 2D NumPy array representing the matrix
    """
    rows, cols = matrix.shape
    for pivot_row in range(min(rows, cols)):
        # Ensure the pivot element is nonzero
        if matrix[pivot_row, pivot_row] == 0:
            non_zero_row = np.argmax(matrix[pivot_row:, pivot_row]) + pivot_row
            row_swap(matrix, pivot_row, non_zero_row)

        # Make the pivot element 1
        pivot_value = matrix[pivot_row, pivot_row]
        row_scale(matrix, pivot_row, 1 / pivot_value)

        # Eliminate nonzero values below the pivot
        for i in range(pivot_row + 1, rows):
            multiplier = matrix[i, pivot_row]
            row_add(matrix, pivot_row, i, -multiplier)

def forward_substitution(matrix, b):
    """
    Perform forward substitution to solve for variables in a lower triangular system.

    Parameters:
    - matrix: Lower triangular matrix (2D NumPy array)
    - b: 1D NumPy array representing the right-hand side of the system
    """
    n = len(b)
    x = np.zeros(n)
    for i in range(n):
        x[i] = b[i] - np.dot(matrix[i, :i], x[:i])
    return x

def back_substitution(matrix, b):
    """
    Perform back substitution to solve for variables in an upper triangular system.

    Parameters:
    - matrix: Upper triangular matrix (2D NumPy array)
    - b: 1D NumPy array representing the right-hand side of the system
    """
    n = len(b)
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(matrix[i, i + 1:], x[i + 1:])) / matrix[i, i]
    return x


In [14]:
import numpy as np

def solve_linear_system_with_lu(A, B):
    """
    Solve a linear system AX = B using LU decomposition.

    Parameters:
    - A: Coefficient matrix (2D NumPy array)
    - B: Right-hand side vector (1D NumPy array)

    Returns:
    - X: Solution vector
    """
    # Step 1: Compute P*B
    PB = np.dot(P, B)

    # Step 2: Solve LY = PB for Y using forward substitution
    Y = forward_substitution(L, PB)

    # Step 3: Solve UX = Y for X using back substitution
    X = back_substitution(U, Y)

    return X

## 3. Obtain the solution for the following system of equations using the LU decomposition code written by you.  

$\begin{aligned} & 2 x_1+5 x_2+0 x_3-4 x_4=\left[\begin{array}{lll}6 & 56 & 9\end{array}\right] \\ & -4 x_1-4 x_2-3 x_3+7 x_4=\left[\begin{array}{lll}36 & 66 & 6\end{array}\right] \\ & -6 x_1-3 x_2-7 x_3-6 x_4=\left[\begin{array}{lll}35 & 58 & -22\end{array}\right] \\ & -x_1+2 x_2-6 x_3+5 x_4=\left[\begin{array}{lll}63 & -14 & 10\end{array}\right] \\ & \end{aligned}$

In [16]:
import numpy as np

# Given system
A = np.array([[2, 5, 0, -4],
              [-4, -4, -3, 7],
              [-6, -3, -7, -6],
              [-1, 2, -6, 5]])

B = np.array([[6, 56, 9],
              [36, 66, 6],
              [35, 58, -22],
              [63, -14, 10]])

# LU decomposition
L, U, P = gaussian_elimination_with_pivot(A)

# Initialize an array to store the solutions
solutions = []

# Solve the system for each right-hand side vector in B
for b_vector in B.T:
    solution_X = solve_linear_system_with_lu(A, b_vector)
    solutions.append(solution_X)

# Display the solutions
for i, solution in enumerate(solutions, 1):
    print(f"Solution {i}:")
    print(solution)
    print()


Solution 1:
[-56199.4375  20374.375   41507.      -2634.75  ]

Solution 2:
[-273844.375   99291.75   202282.     -12835.5  ]

Solution 3:
[-41377.66666667  15002.66666667  30568.          -1940.        ]



## 4. (Jacobi Iteration). 
Suppose that A is a strictly diagonally dominant matrix. Then AX = B has a unique solution X = P . Iteration using the formula $x_j^{(k+1)}=\frac{b_j-a_{j 1} x_1^{(k)}-\cdots-a_{j j-1} x_{j-1}^{(k)}-a_{j j+1} x_{j+1}^{(k)}-\cdots-a_{j N} x_N^{(k)}}{a_{j j}}$ 

for j = 1, 2, . . . , N , will produce a sequence of vectors {Pk } that will converge to P
for any choice of the starting vector P_0 . 
## (Gauss-Seidel Iteration). 
For Gauss-Seidel iteration, the new coordinates are used as they become available. Therefore, the above iteration formula becomes:

$x_j^{(k+1)}=\frac{b_j-a_{j 1} x_1^{(k+1)}-\cdots-a_{j j-1} x_{j-1}^{(k+1)}-a_{j j+1} x_{j+1}^{(k)}-\cdots-a_{j N} x_N^{(k)}}{a_{j j}}$ 

Write codes for both the Jacobi and Gauss-Seidel iteration scheme to obtain the solution
of AX = B iteratively.


In [17]:
import numpy as np

def jacobi_iteration(A, B, initial_guess, max_iterations=100, tolerance=1e-6):
    """
    Solve the system AX = B using Jacobi iteration.

    Parameters:
    - A: Coefficient matrix (2D NumPy array)
    - B: Right-hand side vector (1D NumPy array)
    - initial_guess: Initial guess for the solution vector
    - max_iterations: Maximum number of iterations (default is 100)
    - tolerance: Convergence tolerance (default is 1e-6)

    Returns:
    - solution: Solution vector
    - iterations: Number of iterations performed
    """
    N = len(B)
    x = initial_guess.copy()
    x_new = np.zeros(N)

    for k in range(max_iterations):
        for j in range(N):
            sum_term = np.dot(A[j, :N], x[:N])
            x_new[j] = (B[j] - sum_term + A[j, j] * x[j]) / A[j, j]

        # Check for convergence
        if np.linalg.norm(x_new - x) < tolerance:
            return x_new, k + 1  # Return the solution and the number of iterations

        x = x_new.copy()

    raise ValueError("Jacobi iteration did not converge within the specified number of iterations.")

# Example usage
A = np.array([[10, -1, 2],
              [-1, 11, -1],
              [2, -1, 10]])

B = np.array([6, 25, -11])

initial_guess = np.zeros_like(B)
solution, iterations = jacobi_iteration(A, B, initial_guess)

print("Solution using Jacobi iteration:")
print(solution)
print(f"Number of iterations: {iterations}")


Solution using Jacobi iteration:
[ 1.04326932  2.26923071 -1.08173067]
Number of iterations: 12


In [20]:
import numpy as np

def gauss_seidel_iteration(A, B, initial_guess, max_iterations=100, tolerance=1e-6):
    """
    Solve the system AX = B using Gauss-Seidel iteration.

    Parameters:
    - A: Coefficient matrix (2D NumPy array)
    - B: Right-hand side vector (1D NumPy array)
    - initial_guess: Initial guess for the solution vector
    - max_iterations: Maximum number of iterations (default is 100)
    - tolerance: Convergence tolerance (default is 1e-6)

    Returns:
    - solution: Solution vector
    - iterations: Number of iterations performed
    """
    N = len(B)
    x = initial_guess.copy()

    for k in range(max_iterations):
        for j in range(N):
            sum_term_1 = np.dot(A[j, :j], x[:j])  # Corrected indices for sum_term_1
            sum_term_2 = np.dot(A[j, j+1:], x[j+1:])
            x[j] = (B[j] - sum_term_1 - sum_term_2) / A[j, j]

        # Check for convergence
        if np.linalg.norm(np.dot(A, x) - B) < tolerance:
            return x, k + 1  # Return the solution and the number of iterations

    raise ValueError("Gauss-Seidel iteration did not converge within the specified number of iterations.")

# Example usage
A = np.array([[10, -1, 2],
              [-1, 11, -1],
              [2, -1, 10]])

B = np.array([6, 25, -11])

initial_guess = np.zeros_like(B)
solution, iterations = gauss_seidel_iteration(A, B, initial_guess)

print("Solution using Gauss-Seidel iteration:")
print(solution)
print(f"Number of iterations: {iterations}")


ValueError: Gauss-Seidel iteration did not converge within the specified number of iterations.

In [21]:
import numpy as np

# Example usage with increased max_iterations and tolerance
A = np.array([[10, -1, 2],
              [-1, 11, -1],
              [2, -1, 10]])

B = np.array([6, 25, -11])

initial_guess = np.zeros_like(B)
solution, iterations = gauss_seidel_iteration(A, B, initial_guess, max_iterations=200, tolerance=1e-8)

print("Solution using Gauss-Seidel iteration:")
print(solution)
print(f"Number of iterations: {iterations}")


ValueError: Gauss-Seidel iteration did not converge within the specified number of iterations.

## 5. Solve the following band system using both the Jacobi and Gauss Seidel iteration.

In [22]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Create the banded matrix A
size = 50  # size of the system
diagonals = [[-2] * (size - 1), [12] * size, [-2] * (size - 1)]
A = diags(diagonals, [-1, 0, 1], shape=(size, size)).toarray()

# Create the right-hand side vector B
B = np.ones(size) * 5

# Initial guess for the solution vector
initial_guess = np.zeros(size)

# Jacobi Iteration
solution_jacobi, _ = jacobi_iteration(A, B, initial_guess)

# Gauss-Seidel Iteration
solution_gauss_seidel, _ = gauss_seidel_iteration(A, B, initial_guess)

print("Solution using Jacobi iteration:")
print(solution_jacobi)

print("\nSolution using Gauss-Seidel iteration:")
print(solution_gauss_seidel)


Solution using Jacobi iteration:
[0.51776694 0.6066017  0.62184333 0.62445838 0.62490704 0.62498402
 0.62499722 0.62499949 0.62499988 0.62499994 0.62499995 0.62499996
 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996
 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996
 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996
 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996 0.62499996
 0.62499996 0.62499996 0.62499996 0.62499995 0.62499994 0.62499988
 0.62499949 0.62499722 0.62498402 0.62490704 0.62445838 0.62184333
 0.6066017  0.51776694]

Solution using Gauss-Seidel iteration:
[0.51776694 0.60660171 0.62184334 0.62445839 0.62490706 0.62498404
 0.62499725 0.62499952 0.62499991 0.62499997 0.62499998 0.62499999
 0.62499999 0.62499999 0.62499999 0.62499999 0.62499999 0.62499999
 0.62499999 0.62499999 0.62499999 0.62499999 0.62499999 0.62499999
 0.62499999 0.62499999 0.62499999 0.62499999 0.62499999 0.62499999
 0.62499999 0.62499999 0.6249999